In [ ]:
data = pd.read_csv('sample_data/mnist_train_small.csv')
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)
data_test = data[0:1000].T
X_test = data_test[1:n].T
Y_test = data_test[0].T

data_train = data[1000:m].T 
Y_train = data_train[0].T
X_train = data_train[1:n].T

In [ ]:
!pip install gradio

In [ ]:
import numpy as np
import sympy as sp
import pandas as pd
import gradio

In [ ]:
def one_hot(i, size):
    arr = np.zeros(size)
    arr[i] = 1
    return arr

In [ ]:
def wrap_image(m):
    return m/255

In [ ]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
def mnistize(img):
    img = img.reshape(1, 28, 28, 1)
    img = img.flatten()
    img = np.absolute(img-255)
    return img

In [ ]:
class Layer:
    def __init__(self, layerM, layerN, c):
        self.c = 0.005
        self.Zl = np.empty((layerN, 1), dtype=float)
        self.w = np.random.rand(layerM, layerN) - 0.5
        self.b = np.random.rand(layerM, 1) - 0.5

    def tanh(self,m):
        return np.tanh(m)

    def tanh_diff(self,m):
        return 1 - (np.tanh(m)**2)

    def forward(self, a1):
        self.Zl = (np.matmul(self.w, a1))
        self.Zl = (self.Zl).reshape(10,1)
        self.Zl += self.b
        return self.tanh(self.Zl)

    def computate_weights(self, a1, a2, v):
        dZ = 2*self.tanh_diff(self.Zl) * (a2-v)
        dZ = dZ.T
        dW = np.matmul(a1, dZ)
        dB = dZ
        return dW, dB

    def backward(self, a1, a2, v):
        dW, dB = self.computate_weights(a1, a2, v)
        self.w -= dW.T*self.c
        self.b -= dB.T*self.c


In [ ]:
class Network:
    layers = 1
    def __init__(self, layer_size, n, c):    
        self.layer = Layer(layer_size, n, c)
        self.output = np.empty((10,1), dtype=float)
        self.loss = 0

    def forward_pass(self, a1):
        self.output = self.layer.forward(a1)
        # print("a2 = ", self.output)

    def computate_loss(self, activation, validation):
        return np.sum((activation-validation)**2)

    def backward_pass(self, a1, a2, v):
        self.layer.backward(a1, a2, v)

    def train(self, X_train, Y_train):
        for j in range(10):
            for i in range(X_train.shape[0]):
                self.forward_pass(X_train[i])
                y = one_hot(Y_train[i], 10).reshape(10, 1)
                # print("y = ", y)
                # print("Loss = ", self.computate_loss(self.output, y))
                self.backward_pass(X_train[i].reshape(784,1), self.output, y)

    def testNN(self, X_test, Y_test):
        hit = 0
        for i in range(1000):
            a2 = self.layer.forward(X_test[i])
            # print(a2)
            sm = softmax(a2)
            index = np.argmax(sm)
            if(index == Y_test[i]):
                hit += 1
            # else:
            #     print(Y_test[i], "; meu chute = ", index)
        return (hit * 100)/1000

    def predict(self, img):
        img = mnistize(img)
        a2 = self.layer.forward(img)
        sm = softmax(a2)
        return {str(i): sm[i] for i in range(10)}


In [ ]:
n = Network(10, 784, 0.005)
n.train(X_train, Y_train)

In [ ]:
output_component = gradio.outputs.Label(num_top_classes=3)

/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [ ]:
gradio.Interface(fn=n.predict,
        inputs="sketchpad",
        outputs="label",
        debug=True,
        live=True).launch(share=True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:44: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'debug': True}
  f"You have unused kwarg parameters in {cls}, please remove them: {kwargs}"


Hint: Set streaming=True for Sketchpad component to use live streaming.
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://10283.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f3387aafed0>,
 'http://127.0.0.1:7862/',
 'https://10283.gradio.app')